### Reading a model from file

For troubleshooting purposes it can be useful to save a snapshot of the model to a file. That file will follow a certain format based on the filename suffix. The .lp format is human-readable but does not guarantee full precision. The .mps format is not ideal for human consumption but guarantees no precision loss.

In [1]:
import gurobipy as grb

The global function "read" takes in a filename and returns a Model object.

In [3]:
m = grb.read('diet.lp')
m

<gurobi.Model Continuous instance Unnamed: 2 constrs, 5 vars, Parameter changes: LogFile=gurobi.log>

In [4]:
m.optimize()

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+01, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Presolve removed 0 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.650000e+01   0.000000e+00      0s
       2    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  1.310000000e+02


References to Var and Constr objects are not saved from the previous session, but can be reconstituted from their names by calling Model.getVarByName() and Model.getConstrByName(), respecitvely. We recommend that you follow a consistent naming convention for both variables and constraints so that you will be able to do this in a programmatic way. The convention we follow here is a base name followed by a period-separated list of variable indices.

In [5]:
x1 = m.getVarByName('consumed.1')
x2 = m.getVarByName('consumed.2') 
x3 = m.getVarByName('consumed.3')
x4 = m.getVarByName('consumed.4')
x5 = m.getVarByName('consumed.5')
print x1.X, x2.X, x3.X, x4.X, x5.X

0.0 0.0 0.0 1.0 10.0


### Sensitivity Analysis

We have built and solved the following linear program:
\begin{eqnarray}
\min_x && z = 20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5 \\
\mbox{s.t.} && 2 x_ 1 + 3 x_3 + x_4 + 2 x_5 \ge 21 \\
&& x_2 + 2 x_3 + 2 x_4 + x_5 \ge 12 \\
&& x_i \ge 0,\;\;i = 1,\ldots,5
\end{eqnarray}

and obtained an optimal solution of $x^*_1 = x^*_2 = x^*_3 = 0$, $x^*_4 = 1$, $x^*_5 = 10$ that satisfies the nutrient requirements at a minimum cost of $z^* = 131$. 

We can interpret the right-hand sides of the structural constraints as resource requirements. In the case of the diet problem, the resouces in question are nutrients. Gurobi automatically computes the marginal cost of each resource, which you can access via the Pi attribute of the Constr object.

In [6]:
iron_constr = m.getConstrByName('nutrient.iron')
calcium_constr = m.getConstrByName('nutrient.calcium')
print iron_constr.Pi, calcium_constr.Pi

4.33333333333 3.33333333333


This value is known as the dual value, or shadow price, of the constraint. The interpretation here is that to add an additional unit of iron to your diet (in addition to the 21 units already required) will cost $4 \frac{1}{3}$. Similarly, an additional unit of calcium will cost $3 \frac{1}{3}$. The dual value of a constraint can be thought of as the price of the underlying resource. That is, if someone was selling iron (calcium) supplements, the highest price you'd be willing to pay is $4 \frac{1}{3}$ ($3 \frac{1}{3}$) per unit. Gurobi gives you this sensitivity information effectively for free because it uses the dual values to prove optimality.

More formally, we can view the optimal cost $z^*$ as a function of the objective coefficients, constraint matrix coefficients, and constraint right-hand sides. Let's denote the right hand side of the $i$th constraint as $b_i$. Then the dual value of the $i$th constraint is really just the partial derivative of $z^*$ with respect to $b_i$, that is $\pi^*_i = \frac{\partial{z^*}}{\partial b_i}$.

But, you don't have to take my word for it. We can verify this by actually changing the right-hand side of the constraint and reoptimizing. Each Constr object has a settable attribute RHS that lets you change its right-hand side.

#### Exercise:   Suppose we increase the iron requirement by 0.1 units. What is the optimal cost of the new diet?

In [ ]:
# First predict the optimal cost of the new diet
new_cost = ...
print new_cost

In [ ]:
# Change the RHS of the iron constraint and reoptimize
...
m.optimize()

In [ ]:
print new_cost, m.ObjVal

### Proving Optimality

How exactly does Gurobi use these dual values s to prove optimality? Consider food type 1, which costs 20 per ounce and provides 2 units of iron. The price of iron as implied by the dual value is $4 \frac{1}{3}$. So the effective cost of food type 1 can be computed as:

In [8]:
print 20 - 2*iron_constr.Pi
print x1.RC

11.3333333333
11.3333333333


In [10]:
x1.Obj -= 11.3333
m.optimize()
x1.X

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [9e+00, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.310000000e+02


0.0

In [11]:
x1.Obj -= 0.1
m.optimize()
x1.X

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [9e+00, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.9993333e+29   1.333333e+30   1.999333e-01      0s
       1    1.3025025e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.302502500e+02


7.5

This is the so-called "reduced cost" of $x_1$. The interpretation is that even taking into account the value of the nutrients it provides, food type 1 is overpriced by $11 \frac{1}{3}$.

#### Exercise: Compute the reduced costs of the other four food types.

In [ ]:
# Food type 2 costs 10 per ounce and provides 1 unit of calcium.


In [ ]:
# Food type 3 costs 31 per ounce and provides 3 units of iron and 2 of calcium


In [ ]:
# Food type 4 costs 11 per ounce and provides 1 unit of iron and 2 of calcium


In [ ]:
# Food type 5 costs 12 per ounce and provides 2 units of iron and 1 of calcium


### More on reduced costs

Recall that the constraints in an optimization model are catagorized as structural constraints if they involve functions of more than one variable and as simple bound constraints if they involve only a single variable. The dual values give us sensitivity information on the structural constraints. Specifically, they tell us how much the objective will change if we increase the right-hand side of a constraint by 1. The reduced costs perform the same sensitivity analysis role on the simple bound constraints (in this case, nonnegativity constraints).

For example, the reduced cost of $x_1$ is $11 \frac{1}{3}$. Our optimal diet didn't include food type 1 because it was overpriced. The reduced cost tells us that if we were forced to include at least 1 ounce of food type 1 in our diet, that the cost would increase by $11 \frac{1}{3}$. We can change the lower bound on a decision variable by setting the LB attribute of the corresponding Var object. And, Gurobi has actually already computed the reduced costs for us. They can be accessed via the RC attribute of the Var object.

In [12]:
for var in m.getVars():
    print var.VarName, var.X, var.RC

consumed.1 7.5 0.0
consumed.2 0.0 6.641675
consumed.3 0.0 11.4333
consumed.4 6.0 0.0
consumed.5 0.0 0.074975


#### Exercise: Use the reduced cost of $x_1$ to predict the minimum cost of a diet that was forced to include at least 0.1 ounces of food type 1. Change the lower bound of $x_1$ to 0.1 and reoptimize to verify your calculation.

In [ ]:
# Predict the new cost of the diet
new_cost = ...

In [ ]:
# Adjust the lower bound of x1 and reoptimize
...
m.optimize()

In [ ]:
print new_cost, m.ObjVal

#### Exercise: Restore the bounds and right-hand sides of the model to their original values and reoptimize. Iterate over all variables and for each print out the variable name, optimal value, and reduced cost. What do you observe?